In [25]:
import pandas as pd
import json

In [26]:
with open('./filtered_data.json', 'r') as f:
    data = json.load(f)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(data.items()), columns=['File Name', 'Content'])

In [27]:
len(df)

780

In [28]:
df = df[df['Content'] != "Code NA"]

In [29]:
len(df)

728

In [30]:
# list(df[100:200]["Content"])[:5]

In [ ]:
import base64
from openai import OpenAI

client = OpenAI(
)

In [32]:
merge_snippets_prompt = """
I will provide you a list of code snippets that may contain overlapping or redundant elements. 
Your task is to sequentially process these snippets and merge them into a coherent code file. 
While doing so, ensure that redundancies are eliminated, and overlapping sections are appropriately merged. 
Do not add any additional code beyond what is provided; simply integrate the snippets to create a unified and functional result.
Do not include any comments that say stuff like here is the extracted code, etc. 
It is important that you do not omit any code as well.
Please return only the final merged code block with no other explanation.

Here is the list of code snippets in order: {}
"""

def process_one_batch(code_snippets):
    prompt = merge_snippets_prompt.format(code_snippets)
    messages = [{"role": "user", "content": prompt}]

    response_params = {
        "model": "gpt-4o",
        "messages": messages,
        "temperature": 0,
    }

    response = client.chat.completions.create(**response_params)
    consolidated_code = response.choices[0].message.content
    return consolidated_code

In [34]:
from tqdm import tqdm

df['Frame_ID'] = df['File Name'].str.extract(r'frame_(\d+).jpg').astype(int)

# Set the chunk size and overlap
chunk_size = 600
overlap = 120

# Determine the maximum frame number
max_frame_id = df['Frame_ID'].max()

consolidated_code_list = []

count = 0
# Iterate over the data with an overlap
for start_id in tqdm(range(1, max_frame_id + 1, chunk_size - overlap)):
    end_id = start_id + chunk_size - 1
    subset = df[(df['Frame_ID'] >= start_id) & (df['Frame_ID'] <= end_id)]
    
    # Do whatever processing you need with 'subset'
    # print(f"Processing frames from {start_id} to {end_id}:")
    code_snippets = list(subset["Content"])
    consolidated_code = process_one_batch(code_snippets)
    consolidated_code_list.append(consolidated_code)


100%|██████████| 15/15 [06:07<00:00, 24.52s/it]


In [39]:
file_name = "10min.json"

# Save the list to a file as JSON
with open(file_name, "w") as file:
    json.dump(consolidated_code_list, file)